In [1]:

commit <- "New German Data."
# give a reason for the run

percode <- "2021.Q1"
# Data Collection Code, this controls file paths and output names

run_type <- 1
#run_type =  0 - lite run with no reporting, not recommended.
#run_type =  1 - lite run with normal reporting, default setting.
#run_type =  2 - Heavy run with full reporting, available for audits and troubleshooting.

specialchars <- "-GTHtest"
# optional - add up to a 12 character code in order to mark your instance record .ipynb

#inst_datetime <- datetime.now().strftime("%m%d%Y%H%M%S")
# a single datetime stamp for the full instance run



In [2]:
install.packages("arrow")
install.packages("flexdashboard")
install.packages("openxlsx") # excel
install.packages("data.table") # data manipulation
install.packages("dplyr") # data manipulation used in some of the viz
install.packages("magrittr") # chaining
install.packages("knitr") # html table output with kable function
install.packages("kableExtra") # addtl styling to kable tables
install.packages("ggplot2") # visualization
install.packages("patchwork") # viz, combine plots in one image
install.packages("DT") # html table output (javascript)
install.packages("scales") # plot scales
install.packages("stringr") # string replace
install.packages("reticulate") # execute python chunks


#----- Libraries ------
library(arrow)
library(flexdashboard)
library(openxlsx) # excel
library(data.table) # data manipulation
library(dplyr) # data manipulation used in some of the viz
library(magrittr) # chaining
library(knitr) # html table output with kable function
library(kableExtra) # addtl styling to kable tables
library(ggplot2) # visualization
library(patchwork) # viz, combine plots in one image
library(DT) # html table output (javascript)
library(scales) # plot scales
library(stringr) # string replace
library(reticulate) # execute python chunks




Installing package into 'C:/Users/gharmo/Documents/R/win-library/3.6'
(as 'lib' is unspecified)


package 'arrow' successfully unpacked and MD5 sums checked

The downloaded binary packages are in
	C:\Users\gharmo\AppData\Local\Temp\RtmpW8617z\downloaded_packages


Installing package into 'C:/Users/gharmo/Documents/R/win-library/3.6'
(as 'lib' is unspecified)


package 'flexdashboard' successfully unpacked and MD5 sums checked

The downloaded binary packages are in
	C:\Users\gharmo\AppData\Local\Temp\RtmpW8617z\downloaded_packages


Installing package into 'C:/Users/gharmo/Documents/R/win-library/3.6'
(as 'lib' is unspecified)


...

package 'stringr' successfully unpacked and MD5 sums checked

The downloaded binary packages are in
	C:\Users\gharmo\AppData\Local\Temp\RtmpW8617z\downloaded_packages


Installing package into 'C:/Users/gharmo/Documents/R/win-library/3.6'
(as 'lib' is unspecified)



  There is a binary version available but the source version is later:
           binary source needs_compilation
reticulate   1.19   1.20              TRUE



installing the source package 'reticulate'

Warning message in install.packages("reticulate"):
"installation of package 'reticulate' had non-zero exit status"Warning message:
"package 'arrow' was built under R version 3.6.3"

ERROR: Error: package or namespace load failed for 'arrow' in loadNamespace(i, c(lib.loc, .libPaths()), versionCheck = vI[[i]]):
 namespace 'rlang' 0.4.5 is already loaded, but >= 0.4.6 is required


In [ ]:
rt_path <- file.path('//hecate/Insurance_US/Product Development/Product Management/Global PPM/Reporting/Data Collection/Production', percode )

tempfile <- file.path(rt_path,'2021.Q1.localcur.parquet' )

#file.path(directory, paste(id, ".csv", sep=""))

In [ ]:
input <-  read_parquet( tempfile,  col_select = NULL,
  as_data_frame = TRUE,  props = ParquetArrowReaderProperties$create())

In [ ]:
i_bu <- as.data.table( input)
print(i_bu)

In [ ]:
  dt <- merge(dt, i_bu[, .(BU_CODE, bu_descr = BU_DESCR)], by.x = 'business_unit', by.y = 'BU_CODE', all.x = TRUE)

In [ ]:

# Rename columns to make easier to work with in a function
# Replace "."s with "_"s & make lower case
# replace other patterns not needed in description
fixNames <- function(columns) {
  columns <- columns %>%
    tolower() %>% 
    gsub(pattern = '.', replacement = '_', fixed = TRUE) %>% 
    gsub(pattern = '_(paid_+_ocr_+_ibnr)', replacement = '', fixed = TRUE) %>%
    gsub(pattern = '_(excl__az_tech_fee)', replacement = '', fixed = TRUE) %>%
    gsub(pattern = '_(excl__hq_fees)', replacement = '', fixed = TRUE) %>%
    gsub(pattern = 'persons_involved_in_claims', replacement = 'claimants', fixed = TRUE) %>%
    gsub(pattern = 'units_of_risk', replacement = 'insureds', fixed = TRUE) %>%
    gsub(pattern = 'contribution_margin', replacement = 'cm', fixed = TRUE) %>%
    gsub(pattern = 'number_of', replacement = 'num', fixed = TRUE) %>%
    gsub(pattern = '%', replacement = 'pct', fixed = TRUE) %>%
    gsub(pattern = '(', replacement = '', fixed = TRUE) %>% 
    gsub(pattern = ')', replacement = '', fixed = TRUE) %>% 
    gsub(pattern = '-_', replacement = '', fixed = TRUE) %>%
    gsub(pattern = '+', replacement = '', fixed = TRUE) %>% 
    gsub(pattern = '-', replacement = '_', fixed = TRUE)

  return(columns)
}

In [ ]:


#Function to apply b-partner mapping from input file

cleanData <- function(dt = dt) {
  # Clean unknown partner and sublobs
  dt[product_name %in% c('0', '', '-', 'Not Provided', 'Other','NaN' ), product_name := 'Not Available' ]
  dt[product_id_number %in% c('0', '', '-', 'Not Provided', 'Other','NaN' ), product_id_number := 'Not Available' ]
  dt[business_partner_name %in% c('-', '0', '', 'Other','NaN' , 'all BPs', 'Not Provided') | is.na(business_partner_name),
          business_partner_name := 'Not Available']
    dt[business_partner_id_number %in% c('-', '0', '', 'Other','NaN' , 'all BPs', 'Not Provided') | is.na(business_partner_id_number),
          business_partner_id_number := 'Not Available']

  # Partner names and distribution channel fix - BPs that apply to multiple BUs
  dt <- merge(dt, bp_map[bu == 'Multiple', .(bu_bp_name, bp_map_name, channel)], by.x = 'business_partner_name', by.y = 'bu_bp_name',
              all.x = TRUE)
  
  # Partner names and distribution channel fix - BPs that apply to specific BUs
  dt <- merge(dt, bp_map[bu != 'Multiple', .(bu_bp_name, tmp_bp_name = bp_map_name, tmp_chan = channel, bu)], 
              by.x = c('business_partner_name', 'business_unit'), 
              by.y = c('bu_bp_name', 'bu'), all.x = TRUE)
  
  # combine Multiple BU fixes with Individual BU fixes
  # why? When merging, if there is already a column existing of the same name, a duplicate column is produced. 
  # In this case the Individual BU merge renamed these two new columns and we will remove them once we combine them appropriately.
  dt[!is.na(tmp_bp_name), `:=` (
    bp_map_name = tmp_bp_name,
    channel = tmp_chan)]

  # remove temporary columns
  dt[, `:=` (
    tmp_bp_name = NULL,
    tmp_chan = NULL
  )]
  
  # for records with no mapping, fill with actual data for bp and channel
  dt[is.na(bp_map_name), bp_map_name := business_partner_name]
  dt[is.na(channel), channel := distribution_channel]
  
  # !!! For US, in order to reduce file size, bucket all b-partners with revenue 0 - 250k
  # For CH, bucket all b-partners with revenue <= 50k
  # new b-partner field name = bp_name
  dt_bu_bp <- createMetrics(dt, groupby = c( 'business_unit', 'bp_map_name', 'distribution_channel', 
                                             'region', 'type_of_account'), sel_metrics = 'earned_revenues_net_of_taxes')
  
  dt_bu_bp[, bp_er_group := '> 250 K']
  dt_bu_bp[earned_revenues_net_of_taxes <= 250000, bp_er_group := '<= 250 K']
  dt_bu_bp[earned_revenues_net_of_taxes <= 100000, bp_er_group := '<= 100 K']
  dt_bu_bp[earned_revenues_net_of_taxes <= 50000, bp_er_group := '<= 50 K']
  
  dt <- merge(dt, 
              dt_bu_bp[, c('business_unit', 'bp_map_name', 'distribution_channel', 'type_of_account', 'bp_er_group')], 
              by = c('business_unit', 'bp_map_name', 'distribution_channel', 'type_of_account'), all.x = TRUE)
  
  dt[, bp_name := bp_map_name] # create new b-partner column = mapped column + actual values. Further transformations will be done on this column
  dt[business_unit == 'US' & bp_er_group %in% c('<= 50 K', '<= 100 K'), bp_name := 'US Small Partners (<=100K)']
  dt[business_unit == 'CH' & bp_er_group %in% c('<= 50 K'), bp_name := 'CH Small Partners (<=50K)']
  
  # map "Other" for Channel and SubLoB to "Not Provided"; original distribution_channel column is preserved
  dt[channel %in% c('', 'Other', 'Not Provided'), channel := 'Not Available']
  dt[sub_lob %in% c('', 'Other', 'Not Provided'), sub_lob := 'Not Available']
  
  # bp_name - add specificity. In some cases a BU will have the same partner name like "Standard", but different results by channel
  # when aggregating all of this info together to determine yoy impacts by b-partner, it is helpful to not have duplicated bp names
  # will still preserve bp_name, but instead will create an identifier that concatenates bp name and channel
  dt[, bp_chan := paste0(bp_name, '; ', channel)]
  dt[channel == 'Not Provided', bp_chan := bp_name] #exception when no channel provide (lack of extra information)
  
  # Add less granular channel category
  dt[, channel_hi := channel]
  dt[channel %in% c('Offline Travel Agencies', 'Offline Tour Operators'), channel_hi := 'Offline (Travel Agencies/Tour Operators)']
  dt[channel %in% c('Online Travel Agencies (OTAs)', 'Online Tour Operators'), channel_hi := 'Online (Travel Agencies/Tour Operators)']
  dt[channel %in% c('Carriers - Train, Bus',
                    'Managing General Agents',
                    'Carriers - Cruise, Ferry',
                    'Lodging',
                    'Multiple',
                    'Visa Centers',
                    'Other Niche Travel Market',
                    'Payment Administrators'), channel_hi := 'All Other Channels']
#xx#
  
  # add BU descriptions
  dt <- merge(dt, i_bu[, .(BU_CODE, bu_descr = BU_DESCR)], by.x = 'business_unit', by.y = 'BU_CODE', all.x = TRUE)

  # Add a reporting period and combined date of analysis

  dt[,  month_of_analysis_max := format(as.Date(max(date_of_analysis)), '%Y %b'),  by = business_unit]

  dt[, 
    reporting_period := paste(
    format(as.Date(reporting_date_from), '%Y %b'),
    'to',
    format(as.Date(reporting_date_to), '%Y %b')
    )]

  # Replace LoB column with "Travel" since some show up blank or NA
    dt[, lob := 'Travel']

  # Remove BUs that are not 12 months rolling, but first capture which only reported YTD figures (to capture in BU Detail)
    ytd_bu <- dt[analysed_months != 12, unique(business_unit)] 
    dt <- dt[analysed_months == 12,]

  dt[]
  
}

In [ ]:

d_input <- i_bu

#For a quick run withlimited data, for instance if you are only reviewing a note update
#d_input <- head(d_input,1000)

# necessary type conversion from pandas  #review code #c#
d_input <- d_input  %>% mutate_if(is.list, as.character)


setDT(d_input)

d <- copy(d_input)


# Data manipulations
d <- cleanData(d)